In [93]:

import jieba
import jieba.analyse
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from snownlp import SnowNLP

In [78]:
# 导入训练数据
data = pd.read_excel('feedback.xlsx')

data = np.array(data)

# 获取分类集和数据集
classification = np.array(data[:, 0], dtype='int')
comment = np.array(data[:, 1], dtype='object')

# 对数据集（文本）进行分词
commentStack = [];
for number in range(len(comment)):
    commentWord = jieba.cut(str(comment[number]))
    commentWord = " ".join(commentWord)
    commentStack.append(commentWord)

#转化成数组，格式是["词语1 词语2 词语3"， ...] 一个元素代表一个文本
vectorizer = CountVectorizer()    # 计算文本的词频矩阵
arr = vectorizer.fit_transform(commentStack)  # 矩阵元素a[i][j] 表示j词在i个文本下的词频

transformer = TfidfTransformer()
tfidf = transformer.fit_transform(arr) # 传入词频向量计算tf-idf

In [134]:
# 朴素贝叶斯统计

mnb = MultinomialNB()
mnb.fit(tfidf, classification)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [186]:
from snownlp import sentiment
sentiment.train('feedback_negative.txt', 'feedback_positive.txt')
sentiment.save('sentiment.marshal')

In [188]:
test_comment = u'很棒棒~'

test_comment = jieba.cut(test_comment)
test_comment = " ".join(test_comment)
test_comment = [test_comment]

test_vectorizer_wordFrequency = vectorizer.transform(test_comment)

test_tfidf = transformer.transform(test_vectorizer_wordFrequency).toarray()

predict_classification = mnb.predict(test_tfidf)
predict_classification_proba = mnb.predict_proba(test_tfidf)

print(predict_classification_proba)

snownlp = SnowNLP(str(test_comment))
snow_score = snownlp.sentiments

print(snow_score)

# 如果snow判断大于0.8或朴素贝叶斯预测值大于0.8，则选择作为分类
# snow_tag_stack = {0:0.25, 1:0.5, 2:0.75, 3:1}

naive_score = predict_classification_proba[0][predict_classification[0]]

if (snow_score >= 0.5):
    if (snow_score >= naive_score):      # 如果情感分析得分大于朴素贝叶斯得分，则选择该标签
        if (snow_score > 0.25):
            if (snow_score > 0.5):
                if (snow_score > 0.75):
                    tag = 3
                else:
                    tag = 2
            else:
                tag = 1
        else:
            tag = 0
    else:
        tag = predict_classification[0]
else:
    snow_score = 1-snow_score
    if (snow_score >= naive_score):      # 如果情感分析得分大于朴素贝叶斯得分，则选择该标签
        if (snow_score > 0.5):
            if (snow_score > 0.75):
                tag = 0
            else:
                tag = 1
    else:
        tag = predict_classification[0]
    
print(tag)

[[ 0.05115529  0.28610037  0.14992472  0.51281962]]
0.9990378531689277
3
